In [4]:
import pandas as pd
import sqlite3
import json

#cargaré los datos de los archivos JSON

customers = pd.read_json("sample_analytics.customers.json")
accounts = pd.read_json("sample_analytics.accounts.json")

#Cargar y expandir el JSON de transacciones ya que se encuentra anidado

with open("sample_analytics.transactions.json") as f:
  raw_transactions = json.load(f)

#Vamos a extraer y normalizar las transacciones por cuenta

transactions_expanded = []
for entry in raw_transactions:
    account_id = entry['account_id'] # id de cuenta que agrupa cada set de transacciones
    for tx in entry['transactions']:
        tx_flat = tx.copy()
        tx_flat['transactions_expanded'] = account_id
        if isinstance(tx['date'], dict): #con esto revisamos si la fecha esta en formato {'$date': ...}
            tx_flat['date'] = tx['date']['$date']
        transactions_expanded.append(tx_flat)



In [6]:
#crear dataframe de transacciones y limpiar la data

transactions = pd.DataFrame(transactions_expanded)
transactions['date'] = pd.to_datetime(transactions['date'], errors= 'coerce') #convertir a datetime
transactions = transactions[['transactions_expanded', 'date', 'amount', 'transaction_code', 'symbol', 'price', 'total']]
transactions.head()

,transactions_expanded,date,amount,transaction_code,symbol,price,total
0,852986,2010-02-24 00:00:00+00:00,866,sell,nvda,15.4156878858997501424710208084434270858764648...,13349.98570918918362337990402
1,852986,1990-06-18 00:00:00+00:00,2918,buy,adbe,2.20295603461750477691794003476388752460479736...,6428.225709013878939046549021
2,852986,2006-02-07 00:00:00+00:00,6009,sell,msft,22.6192335814292739826214528875425457954406738...,135918.9745908085073615723104
3,771641,2012-03-14 00:00:00+00:00,2931,buy,sap,65.0757645452902835359054733999073505401611328125,190737.0658822458210437389425
4,771641,2000-09-01 00:00:00+00:00,8077,sell,nvda,12.4174214603589625482982228277251124382019042...,100295.5131353193405026047458
